In [1]:
import numpy as np
import math 

import timeit

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib import colors

from astropy.io import fits

import os

from skyfield.api import load, Loader, wgs84
ts = load.timescale()
import datetime

from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

In [2]:
#define global parameters 
nms = wgs84.latlon(32.9024, -105.5301, elevation_m=2225)

In [3]:
#define intersection algorithm 
#takes an input argument tuple st p1 = (x, y) 
def intersect(p1, p2, p3, p4):
    def subtract(p1, p2):
        return (p1[0] - p2[0], p1[1] - p2[1])

    # calculates the cross product of vector p1 and p2
    # if p1 is clockwise from p2 wrt origin then it returns +ve value
    # if p2 is anti-clockwise from p2 wrt origin then it returns -ve value
    # if p1 p2 and origin are collinear then it returs 0
    def cross_product(p1, p2):
        return p1[0] * p2[1] - p2[0] * p1[1]
    
    # returns the cross product of vector p1p3 and p1p2
    # if p1p3 is clockwise from p1p2 it returns +ve value
    # if p1p3 is anti-clockwise from p1p2 it returns -ve value
    # if p1 p2 and p3 are collinear it returns 0
    def direction(p1, p2, p3):
        return  cross_product(subtract(p3,p1), subtract(p2,p1))
    
    # checks if p lies on the segment p1p2
    def on_segment(p1, p2, p):
        return min(p1[0], p2[0]) <= p[0] <= max(p1[0], p2[0]) and min(p1[1], p2[1]) <= p[1] <= max(p1[1], p2[1])
    
    d1 = direction(p3, p4, p1)
    d2 = direction(p3, p4, p2)
    d3 = direction(p1, p2, p3)
    d4 = direction(p1, p2, p4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and \
        ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True

    elif d1 == 0 and on_segment(p3, p4, p1):
        return True
    elif d2 == 0 and on_segment(p3, p4, p2):
        return True
    elif d3 == 0 and on_segment(p1, p2, p3):
        return True
    elif d4 == 0 and on_segment(p1, p2, p4):
        return True
    else:
        return False
    
#takes 3d velocity vector input and outputs 1d speed value.
def speed(vel):
    s = np.sqrt(vel[0]**2+vel[1]**2+vel[2]**2)
    return s

def sunlit(sats_found, fits_filename, tle_filename): 
    folder_fits = '/Users/maddynardin/Desktop/dragonfly_data/lights_mar27'
    path_fits = os.path.join(folder_fits, fits_filename)
    hdul = fits.open(path_fits)
    image = hdul[0].data
    hdr = hdul[0].header 
    wcs = WCS(hdr)
    tt = hdr['DATE']
    date = datetime.datetime.strptime(tt, '%Y-%m-%dT%H:%M:%S.%f')
    #define time interval (straddel end time by +/- 10 mins)
    exposure = ts.utc(date.year,date.month,date.day,date.hour,date.minute+np.linspace(0,10,1200),date.second)
    
    folder_tle = '/Users/maddynardin/Desktop/dragonfly_data'
    path_tle = os.path.join(folder_tle,tle_filename)
    satellites = load.tle_file(path_tle)
    
    load_e = Loader('~/Documents/fishing/SkyData')  # avoid duplicating large DE files
    eph = load_e('de421.bsp')
    
    plot_these = []
    by_name = {sat.name: sat for sat in satellites}
    for name in by_name:
        satellite = by_name[name]
        
        if name in sats_found:
            plot_these.append(satellite) 
    sun_lit = []  
    for sat in plot_these:
        sunlit = sat.at(exposure).is_sunlit(eph) 
        plt.figure(figsize = (6,2))
        plt.plot(np.linspace(0,10,1200),sunlit, '-k')
        plt.title(str(sat.name)+' is sunlit (boolean output)', fontsize=12)
        plt.xlabel('Minutes Throughout Exposure')
        plt.show()
        
        if any(sunlit) == True: sun_lit.append(str(sat.name))
            
    return sun_lit

def plot_sunlit_intersections(sun_lit,fits_filename, tle_filename):
    folder_fits = '/Users/maddynardin/Desktop/dragonfly_data/lights_mar27'
    path_fits = os.path.join(folder_fits, fits_filename)
    hdul = fits.open(path_fits)
    image = hdul[0].data
    hdr = hdul[0].header 
    wcs = WCS(hdr)
    tt = hdr['DATE']
    date = datetime.datetime.strptime(tt, '%Y-%m-%dT%H:%M:%S.%f')
    #define time interval (straddel end time by +/- 10 mins)
    time_int = ts.utc(date.year,date.month,date.day,date.hour,date.minute+np.linspace(-10,10,1200),date.second)
    
    #define corners of fits file in pixel coordinates
    naxis1 = hdr['NAXIS1']
    naxis2 = hdr['NAXIS2']
    bottom_right = np.array([naxis1,0])
    bottom_left = [0, 0]
    top_right = [naxis1,naxis2]
    top_left = [0,naxis2]
    folder_tle = '/Users/maddynardin/Desktop/dragonfly_data'
    path_tle = os.path.join(folder_tle,tle_filename)
    satellites = load.tle_file(path_tle)

    #symlognorm = colors.SymLogNorm(linthresh=100, linscale=0.5, vmin=-185, vmax=900, base=np.e)
    scale_min = -50 + 10000
    scale_max = 500 + 10000
    symlognorm = colors.LogNorm(vmin=scale_min, vmax=scale_max)
    fig, ax = plt.subplots(dpi=400)
    ax.imshow(image+10000, cmap=cm.gray_r, norm=symlognorm, origin='lower')

    if sun_lit != []:
        plot_these = []
        by_name = {sat.name: sat for sat in satellites}
        for name in by_name:
            satellite = by_name[name]

            if name in sun_lit:
                plot_these.append(satellite) 
                
        for sat in plot_these:  
            #where will the stellite be in reference to palomar
            difference = sat - nms

            ra_list = []
            dec_list = []
            skycoord_list = []

            ra,dec, _ = difference.at(time_int).radec()

            #define cener of image
            approx_center = (3833,2125)
            c_ra,c_dec = wcs.wcs_pix2world(3833,2125,1)
            image_obj = SkyCoord(c_ra,c_dec, unit="deg")

            skycoords = SkyCoord(ra._degrees, dec.degrees, unit='deg')
            nearby = skycoords[skycoords.separation(image_obj).to(u.arcmin).value < 250]
            x, y = wcs.all_world2pix(nearby.ra.deg, nearby.dec.deg, 1)

            ax.plot(x,y, linestyle = 'solid', linewidth = 0.5, label = sat.name)
            #ax.annotate(sat,(x[2],y[2]),(x[2]+400,y[2]),bbox=dict(boxstyle='round',alpha=0.1))
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            #ax.grid(alpha = 0.4)
            ax.set_xlabel('x [pixels]')
            ax.set_ylabel('y [pixels]')
            ax.set_title(fits_filename +' Sunlit Intersections', fontsize = 10)
            ax.set_xlim(0-50,naxis1+50)
            ax.set_ylim(0-50,naxis2+50)
            plt.savefig('final', bbox_inches='tight')
    

In [4]:
def find_sats(fits_filename, tle_filename):
    folder_fits = '/Users/maddynardin/Desktop/dragonfly_data/lights_mar27'
    path_fits = os.path.join(folder_fits, fits_filename)
    hdul = fits.open(path_fits)
    image = hdul[0].data
    hdr = hdul[0].header 
    wcs = WCS(hdr)
    tt = hdr['DATE']
    print(tt)
    date = datetime.datetime.strptime(tt, '%Y-%m-%dT%H:%M:%S.%f')
    #define time interval (straddel end time by +/- 10 mins)
    time_int = ts.utc(date.year,date.month,date.day,date.hour,date.minute+np.linspace(-10,10,1200),date.second)
    
    #define corners of fits file in pixel coordinates
    naxis1 = hdr['NAXIS1']
    naxis2 = hdr['NAXIS2']
    bottom_right = np.array([naxis1,0])
    bottom_left = [0, 0]
    top_right = [naxis1,naxis2]
    top_left = [0,naxis2]
    folder_tle = '/Users/maddynardin/Desktop/dragonfly_data'
    path_tle = os.path.join(folder_tle,tle_filename)
    satellites = load.tle_file(path_tle)

    #symlognorm = colors.SymLogNorm(linthresh=100, linscale=0.5, vmin=-185, vmax=900, base=np.e)
    scale_min = -50 + 10000
    scale_max = 500 + 10000
    symlognorm = colors.LogNorm(vmin=scale_min, vmax=scale_max)
    fig, ax = plt.subplots(dpi=400)
    #ax.imshow(image, norm = symlognorm, origin = 'lower')
    ax.imshow(image+10000, cmap=cm.gray_r, norm=symlognorm, origin='lower')
    
    sats_found = []
    intersections = []
    by_name = {sat.name: sat for sat in satellites}
    for name in by_name:
        satellite = by_name[name]
        #where will the stellite be in reference to palomar
        difference = satellite - nms

        ra_list = []
        dec_list = []
        skycoord_list = []

        ra,dec, _ = difference.at(time_int).radec()

        #define cener of image
        approx_center = (3833,2125)
        c_ra,c_dec = wcs.wcs_pix2world(3833,2125,1)
        image_obj = SkyCoord(c_ra,c_dec, unit="deg")

        skycoords = SkyCoord(ra._degrees, dec.degrees, unit='deg')
        nearby = skycoords[skycoords.separation(image_obj).to(u.arcmin).value < 250]
        x, y = wcs.all_world2pix(nearby.ra.deg, nearby.dec.deg, 1)


        ax.plot(x,y, linestyle = 'solid', linewidth = 0.5)
        ax.set_title(fits_filename +' All Satellites within 250 arcmin', fontsize = 10)
        
        pixel_coords = list(zip(x,y))
        
        if pixel_coords != []: 
            s = (pixel_coords[0], pixel_coords[len(pixel_coords)-1])
            def sat_in_footprint():
                #for s in segments:
                #intersection loop top left to top right
                if intersect(s[0],s[1],top_left,top_right) == True:
                    return s[0], s[1]
                #intersection loop top right to bottom right
                if intersect(s[0],s[1],top_right,bottom_right) == True:
                    return s[0], s[1] 
                #intersection loop bottom right to bottom left
                if intersect(s[0],s[1],bottom_right,bottom_left) == True:
                    return s[0], s[1]
                #intersection loop bottom left to top left
                if intersect(s[0],s[1],bottom_left,top_left) == True:
                    return s[0], s[1]
                return None
            
            intersection = sat_in_footprint()
            if intersection is not None:
                sats_found.append(str(name))
                intersections.append(intersection)
                
    sun_lit_sats = sunlit(sats_found, fits_filename, tle_filename)
    
    plot_sunlit_intersections(sun_lit_sats,fits_filename, tle_filename)
    
    return sats_found, intersections, sun_lit_sats
    